In [1]:
import pandas as pd
df = pd.read_csv('./data/final_dataset.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees
0,6194,255,1,0.00001,0.000011,-1,0,4,0,4,0,0
1,6194,980,1,0.00001,0.000012,-1,0,4,0,15,0,1
2,6194,2992,1,0.00001,0.000014,4,0,4,3,0,0,0
3,6194,2507,1,0.00001,0.000026,3,0,4,8,14,0,0
4,6194,986,1,0.00001,0.000022,10,0,4,1,20,0,0


In [2]:
# standarlize
x = df['Page_Rank_Src']
df['Page_Rank_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Page_Rank_Dst']
df['Page_Rank_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Shortest_Path']
df['Shortest_Path'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followees_Dst']
df['Followees_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followees_Src']
df['Followees_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followers_Dst']
df['Followers_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followers_Src']
df['Followers_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Int_Followees']
df['Int_Followees'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Int_Followers']
df['Int_Followers'] = ( x-x.min()) / ( x.max() - x.min() )
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees
0,6194,255,1,0.0,0.000557,0.00,0.0,0.009901,0.000000,0.009901,0.0,0.000000
1,6194,980,1,0.0,0.000921,0.00,0.0,0.009901,0.000000,0.037129,0.0,0.013158
2,6194,2992,1,0.0,0.001796,0.25,0.0,0.009901,0.007673,0.000000,0.0,0.000000
3,6194,2507,1,0.0,0.008523,0.20,0.0,0.009901,0.020460,0.034653,0.0,0.000000
4,6194,986,1,0.0,0.006426,0.55,0.0,0.009901,0.002558,0.049505,0.0,0.000000


In [3]:
from sklearn.model_selection import train_test_split
df_x = df.drop(columns=['Source', 'Destination', 'Class'])
df_y = df[['Class']]
x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = 0.3, random_state=42)

In [4]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

estimators = [50, 100, 150]
depths = [9, 12, 15, 24]
split = np.random.randint(100, 150, 2)
leaves = np.random.randint(20, 30, 2)
param_grid = {'n_estimators' : estimators, 'max_depth' : depths, 'min_samples_split' : split, 'min_samples_leaf' : leaves}

gs_rf = GridSearchCV(cv = 10, estimator = RandomForestClassifier(), param_grid = param_grid)
gs_rf.fit(x_train, y_train.values.ravel())

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [9, 12, 15, 24],
                         'min_samples_leaf': array([21, 26]),
                         'min_samples_split': array([134, 115]),
                         'n_estimators': [50, 100, 150]})

In [5]:
import pickle

pickle.dump(gs_rf.best_estimator_, open('./data/rf_model.p', 'wb')) 
print(gs_rf.best_params_)

{'max_depth': 24, 'min_samples_leaf': 21, 'min_samples_split': 115, 'n_estimators': 50}


In [6]:
from sklearn.metrics import classification_report
print(classification_report(gs_rf.best_estimator_.predict(x_test), y_test)) 

              precision    recall  f1-score   support

           0       0.98      0.96      0.97     10795
           1       0.96      0.98      0.97     10400

    accuracy                           0.97     21195
   macro avg       0.97      0.97      0.97     21195
weighted avg       0.97      0.97      0.97     21195

